In [9]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, LeakyReLU, Flatten, Dense, Dropout, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix


In [10]:
# Define dataset root directory
dataset_root = "/kaggle/input/indian-currency-dataset/Indian Currency Dataset"

# Define paths for training and testing
train_dir = os.path.join(dataset_root, 'train')
test_dir = os.path.join(dataset_root, 'test')


In [11]:
# Data augmentation for training & validation
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # Use 20% of train data for validation
)

# Load training data (80% of original train data)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(128, 128),
    batch_size=16,
    class_mode='binary',
    subset='training'  # Training set (80%)
)

# Load validation data (20% of original train data)
val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(128, 128),
    batch_size=16,
    class_mode='binary',
    subset='validation'  # Validation set (20%)
)

# Load test data (No split, only rescale)
test_datagen = ImageDataGenerator(rescale=1.0/255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(128, 128),
    batch_size=16,
    class_mode='binary',
    shuffle=False  # Ensure correct label-order mapping
)


Found 199 images belonging to 2 classes.
Found 49 images belonging to 2 classes.
Found 107 images belonging to 2 classes.


In [14]:
def build_discriminator():
    model = Sequential([
        # First Conv Block
        Conv2D(32, (3, 3), padding='same', input_shape=(128, 128, 3)),
        BatchNormalization(),
        LeakyReLU(0.2),
        MaxPooling2D(pool_size=(2,2)), 

        # Second Conv Block
        Conv2D(64, (3, 3), padding='same'),
        BatchNormalization(),
        LeakyReLU(0.2),
        MaxPooling2D(pool_size=(2,2)),

        # Third Conv Block
        Conv2D(128, (3, 3), padding='same'),
        BatchNormalization(),
        LeakyReLU(0.2),
        MaxPooling2D(pool_size=(2,2)),

        # Fully Connected Layers
        Flatten(),
        Dense(128),  
        LeakyReLU(0.2),
        Dropout(0.4),
        Dense(1, activation='sigmoid')  # Binary classification
    ])
    
    return model

# Build and compile the Discriminator model
discriminator = build_discriminator()
discriminator.compile(optimizer=Adam(learning_rate=0.0002),
                      loss='binary_crossentropy',
                      metrics=['accuracy'])

# Display model summary
discriminator.summary()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)                    │ (None, 128, 128, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_6                │ (None, 128, 128, 32)        │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_8 (LeakyReLU)            │ (None, 128, 128, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_6 (MaxPooling2D)       │ (None, 64, 64, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_7 (Conv2D)                    │ (None, 64, 64, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_7                │ (None, 64, 64, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_9 (LeakyReLU)            │ (None, 64, 64, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_7 (MaxPooling2D)       │ (None, 32, 32, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_8 (Conv2D)                    │ (None, 32, 32, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_8                │ (None, 32, 32, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_10 (LeakyReLU)           │ (None, 32, 32, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_8 (MaxPooling2D)       │ (None, 16, 16, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (None, 32768)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 128)                 │       4,194,432 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_11 (LeakyReLU)           │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,288,705 (16.36 MB)

 Trainable params: 4,288,257 (16.36 MB)

 Non-trainable params: 448 (1.75 KB)

In [15]:
# Define checkpoint to save the best model
checkpoint = ModelCheckpoint(
    'best_simplified_discriminator.keras',
    monitor='val_accuracy',
    save_best_only=True,
    verbose=1
)


In [16]:
# Train the model with validation data from train set
history = discriminator.fit(
    train_generator,
    validation_data=val_generator,  # Use validation split
    epochs=100,
    callbacks=[checkpoint],
    verbose=1
)


Epoch 1/100
10/13 ━━━━━━━━━━━━━━━━━━━━ 1s 360ms/step - accuracy: 0.5556 - loss: 1.9549
Epoch 1: val_accuracy improved from -inf to 0.48980, saving model to best_simplified_discriminator.keras
13/13 ━━━━━━━━━━━━━━━━━━━━ 14s 514ms/step - accuracy: 0.5655 - loss: 1.9277 - val_accuracy: 0.4898 - val_loss: 0.7009
Epoch 2/100
10/13 ━━━━━━━━━━━━━━━━━━━━ 0s 179ms/step - accuracy: 0.5739 - loss: 1.8398
Epoch 2: val_accuracy improved from 0.48980 to 0.85714, saving model to best_simplified_discriminator.keras
13/13 ━━━━━━━━━━━━━━━━━━━━ 7s 278ms/step - accuracy: 0.5903 - loss: 1.7515 - val_accuracy: 0.8571 - val_loss: 0.6425
Epoch 3/100
10/13 ━━━━━━━━━━━━━━━━━━━━ 0s 177ms/step - accuracy: 0.5687 - loss: 1.5380
Epoch 3: val_accuracy did not improve from 0.85714
13/13 ━━━━━━━━━━━━━━━━━━━━ 6s 242ms/step - accuracy: 0.5795 - loss: 1.5051 - val_accuracy: 0.4898 - val_loss: 0.7068
Epoch 4/100
10/13 ━━━━━━━━━━━━━━━━━━━━ 0s 208ms/step - accuracy: 0.6512 - loss: 1.0768
Epoch 4: val_accuracy did not improv

In [12]:
# Load the best saved model
best_model = tf.keras.models.load_model('best_simplified_discriminator.keras')

# Evaluate on the test dataset
test_loss, test_accuracy = best_model.evaluate(test_generator, verbose=1)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")
print(f"Test Loss: {test_loss:.4f}")


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 452ms/step - accuracy: 0.8855 - loss: 0.4353
Test Accuracy: 94.39%
Test Loss: 0.2374


In [19]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
import os

# Get predictions (1 = Real, 0 = Fake)
predictions = (best_model.predict(test_generator) > 0.5).astype(int)
true_labels = test_generator.classes
filenames = test_generator.filenames  # Get image filenames

# Classification report
print("\nClassification Report:\n")
print(classification_report(true_labels, predictions, target_names=test_generator.class_indices.keys()))

# Confusion matrix
print("\nConfusion Matrix:\n")
conf_matrix = confusion_matrix(true_labels, predictions)
print(conf_matrix)


7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 369ms/step

Classification Report:

              precision    recall  f1-score   support

        fake       1.00      0.90      0.95        59
        real       0.89      1.00      0.94        48

    accuracy                           0.94       107
   macro avg       0.94      0.95      0.94       107
weighted avg       0.95      0.94      0.94       107


Confusion Matrix:

[[53  6]
 [ 0 48]]
